In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import drive    #montar nuestro Drive en la máquina 

drive.mount('/content/drive')  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
  #cargamos dataset desde drive


df=pd.read_csv('/content/drive/My Drive/datasets csv/movie_data.csv')

In [0]:
from google.colab import files    #Cargamos dataset  local, en este caso no.

files.upload()  

In [23]:
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [0]:
import numpy as np              #Transformamos palabras en vectores de características, modelo <bolsa de palabras> 
from sklearn.feature_extraction.text   import CountVectorizer

count = CountVectorizer()
docs = np.array([
       'The sun is shining',
       'The wather is sweet',
       'The sun is shining, the wather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [25]:
print (count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'wather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [26]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [27]:
from sklearn.feature_extraction.text import TfidfTransformer      #Relevancia de palabras mediante frecuencia de término-frecuencia inversa de documento

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [28]:
df.loc[0, 'review'] [-50:]   #Limpiar datos textuales. Vamos a mostrar los últimos 50 caracteres del primer documento que contienen marcadores HTML, puntuación y otros caracteres no alfabéticos

'is seven.<br /><br />Title (Brazil): Not Available'

In [0]:
import re      #hacemos función q elimina los marcadores HTML y todas las marcas de puntuación excepto la de los emoticonos como :) porque son útiles para el análisis de sentimiento. 
               #usamos la librería de Python regular expression (regex), re, como se muestra a continuación:
                
def preprocessor (text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [30]:
preprocessor(df.loc[0, 'review'] [-50:])   #Vamos a comprobar si nuestro preprocesador funciona correctamente

'is seven title brazil not available'

In [31]:
preprocessor("</a> This :) is :( a test :) !")   

' this is a test :) :( :)'

In [0]:
df['review'] = df['review'].apply(preprocessor)

In [0]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):              #Procesar documentos en componentes léxicos
    return text.split()           #Dividimos en palabras individuales dividiendo a su vez los documentos limpios en espacios en blanco
  

def tokenizer_porter(text):       #la técnica DECLINACIÓN DE PALABRAS es el proceso de transformar una palabra en su forma de raíz. Esto nos permite mapear palabras relacionadas con la misma raíz.
    return [porter.stem(word) for word in text.split()]  

In [34]:
tokenizer('runners like runing and thus they run')  

['runners', 'like', 'runing', 'and', 'thus', 'they', 'run']

In [35]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [36]:
import nltk          #La ELMINACIÓN DE PALABRAS VACÍAS son aquellas que son extremadamente comunes en todo tipo de textos y que no contienen información útil. Ej: "ha, es, y, como"

nltk.download('stopwords')  #Utilizaremos 127 palabras vacías inglesas disponible en la libreria NLTK

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
from nltk.corpus import stopwords   #Despues de descargar el conjunto de palabras vacías, podemos cargar y aplicar este conjunto de palabras vacías inglesas del modo siguiente:

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [0]:
X_train = df.loc[:25000, 'review'].values           #vamos a entrenar un modelo de REGRESIÓN LOGÍSTICA para clasificacion de documentos
y_train = df.loc[:25000, 'sentiment'].values        #Dividimos el dataframe en 25000 documentos para entrenar y 25000 para testear.   
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [0]:
from sklearn.pipeline import Pipeline    #Utilizamos el objeto GRIDSEARCHCV para encontrar el conjunto de parámetros óptimo para nuestro modelo mediante una validación cruzada de 5 iteraciones:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [40]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 37.6min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 174.6min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 220.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                         

In [41]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)  #Una vez acabada la búsqueda de cuadrícula, imprimimos el mejor conjunto de parámetros

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f078ee818c8>} 


In [42]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)     #Con el mejor modelo surgido de la búsqueda de cuadrícula, imprimimos la puntuación de la precisión de la validación cruzada de 5 iteraciones

CV Accuracy: 0.897


In [43]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))   #Imprimimos la precisión de la clasificación en el conjunto de datos de testeo

Test Accuracy: 0.899


Los resultados revelan que nuestro modelo de aprendizaje automático puede predecir si una crítica de cine es positivo o negativa con un 90% de precisión.